In [17]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import numpy as np

# --- 1. Creiamo i dati di esempio con Pandas ---

# Tabella 'funds_master'
funds_data = {
    'fund_id': ['FUND_A', 'FUND_B', 'FUND_C', 'FUND_D', 'FUND_E'],
    'fund_name': ['Alpha Generator', 'Beta Tracker', 'Global Bond', 'Euro Value', 'Stable Growth'],
    'portfolio_manager': ['Anna Rossi', 'Anna Rossi', 'Marco Bianchi', 'Marco Bianchi', 'Luca Verdi']
}
df_funds = pd.DataFrame(funds_data)

# Tabella 'daily_returns'
# Creiamo 3 mesi di dati per 5 fondi
dates = pd.to_datetime(pd.date_range(end=datetime.now(), periods=90, freq='B'))
funds = ['FUND_A', 'FUND_B', 'FUND_C', 'FUND_D', 'FUND_E']
all_returns = []

for fund in funds:
    # Generiamo rendimenti casuali per ogni fondo
    np.random.seed(abs(hash(fund)) % (10**8)) # Seed per rendimenti consistenti
    returns = np.random.normal(loc=0.0005, scale=0.01, size=len(dates))
    fund_returns = pd.DataFrame({
        'report_date': dates,
        'fund_id': fund,
        'daily_return': returns
    })
    all_returns.append(fund_returns)

df_returns = pd.concat(all_returns)



# --- 2. Creiamo un database in-memory e carichiamo le tabelle ---

conn = sqlite3.connect(':memory:')
df_funds.to_sql('funds_master', conn, index=False)
df_returns.to_sql('daily_returns', conn, index=False)
print(df_returns)
print(df_funds)
print("Database e tabelle di test create. Pronto per eseguire la query.")



                  report_date fund_id  daily_return
0  2025-04-21 18:43:18.662344  FUND_A      0.011432
1  2025-04-22 18:43:18.662344  FUND_A     -0.005319
2  2025-04-23 18:43:18.662344  FUND_A      0.011085
3  2025-04-24 18:43:18.662344  FUND_A     -0.001557
4  2025-04-25 18:43:18.662344  FUND_A      0.007190
..                        ...     ...           ...
85 2025-08-18 18:43:18.662344  FUND_E     -0.005121
86 2025-08-19 18:43:18.662344  FUND_E      0.015326
87 2025-08-20 18:43:18.662344  FUND_E     -0.005603
88 2025-08-21 18:43:18.662344  FUND_E      0.010041
89 2025-08-22 18:43:18.662344  FUND_E     -0.005930

[450 rows x 3 columns]
  fund_id        fund_name portfolio_manager
0  FUND_A  Alpha Generator        Anna Rossi
1  FUND_B     Beta Tracker        Anna Rossi
2  FUND_C      Global Bond     Marco Bianchi
3  FUND_D       Euro Value     Marco Bianchi
4  FUND_E    Stable Growth        Luca Verdi
Database e tabelle di test create. Pronto per eseguire la query.


In [18]:
# --- 3. Esegui la tua query qui ---

# Definiamo l'ultimo mese come i 30 giorni precedenti a oggi
last_month_start_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')

## ---------------------------------------------------------------------------
## SCRIVI LA TUA QUERY QUI
## Sostituisci il contenuto della stringa 'query' con la tua soluzione.
## ---------------------------------------------------------------------------
query = f"""

WITH
    global_returns 
AS (SELECT 
        fund_id, SUM(daily_return) AS month_return
    FROM
        daily_returns
    WHERE
         report_date >= '2025-07-25'
    GROUP BY 
        fund_id
    )


SELECT *, DENSE_RANK() OVER ( PARTITION BY funds_master.portfolio_manager ORDER BY global_returns.month_return DESC) AS 'rank'
FROM global_returns JOIN funds_master ON global_returns.fund_id = funds_master.fund_id
ORDER BY funds_master.portfolio_manager, rank;

"""

# Esegui la query sul database in memoria
try:
    result_df = pd.read_sql_query(query, conn)
    print("\\n--- Risultato della Tua Query ---")
    display(result_df)
except Exception as e:
    print(f"\\nErrore nell'esecuzione della query: {e}")

# --- 4. Chiudi la connessione ---
conn.close()

\n--- Risultato della Tua Query ---


,fund_id,month_return,fund_id,fund_name,portfolio_manager,rank
0,FUND_A,0.042178,FUND_A,Alpha Generator,Anna Rossi,1
1,FUND_B,-0.002946,FUND_B,Beta Tracker,Anna Rossi,2
2,FUND_E,0.000332,FUND_E,Stable Growth,Luca Verdi,1
3,FUND_D,0.102612,FUND_D,Euro Value,Marco Bianchi,1
4,FUND_C,-0.006502,FUND_C,Global Bond,Marco Bianchi,2
